In [1]:
import pandas as pd
import numpy as np
import datetime

data = pd.read_excel('last Ю3 (1).xlsx', sheet_name='ГРП')
dummy = pd.read_excel('last Ю3 (1).xlsx', sheet_name='МЭР')
dummy.head(20)

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт
0,2005-04-01,1,NaN,0.0,NaN,NaN,NaN,NaN
1,2005-05-01,1,NaN,0.0,NaN,NaN,NaN,NaN
2,2005-06-01,1,NaN,0.0,NaN,NaN,NaN,NaN
3,2005-07-01,1,NaN,0.0,NaN,NaN,NaN,NaN
4,2005-08-01,1,NaN,0.0,NaN,NaN,NaN,NaN
5,2005-09-01,1,NaN,0.0,NaN,NaN,NaN,NaN
6,2005-10-01,1,NaN,0.0,NaN,NaN,NaN,NaN
7,2005-11-01,1,NaN,0.0,NaN,NaN,NaN,NaN
8,2005-12-01,1,NaN,0.0,NaN,NaN,NaN,NaN
9,2006-01-01,1,NaN,0.0,NaN,NaN,NaN,NaN


In [2]:
date_column_name = 'Дата ВНР после'
target = 'Скважина №'
data['Скважина №'] = data.apply(
            lambda x:
            str(x['Скважина №']), axis=1)
dummy['Скважина №'] = dummy.apply(
            lambda x:
            str(x['Скважина №']), axis=1)
print(data.shape, dummy.shape, date_column_name)

(863, 11) (315120, 8) Дата ВНР после


In [3]:
dummy['Index'] = dummy.index
dummy1 = dummy.drop(dummy[dummy['Пластовое давление (ТР), атм'] != 0].index)
dummy1 = dummy1.drop(dummy1[dummy1['Забойное давление (ТР), атм'] != 0].index)
dummy1 = dummy1.groupby('Скважина №').agg(
                {'Index': lambda x:
                x.tolist()})
dummy1['кол'] = dummy1.apply(lambda x: len(x['Index']), axis=1)
dummy1 = dummy1.drop(dummy1[dummy1['кол'] < 3].index)

delete = []
for i in dummy1['Index']:
    delete += i
dummy = dummy[~dummy['Index'].isin(delete)].reset_index(drop=True)

dummy2 = dummy[dummy['Забойное давление (ТР), атм'].isna()]
dummy2 = dummy2.groupby('Скважина №').agg(
                {'Index': lambda x:
                x.tolist()})
dummy2['забойки'] = dummy2.apply(lambda x: len(x['Index']), axis=1)
dummy2 = dummy2.drop(dummy2[dummy2['забойки'] < 3].index)
            # wells_del_both=dummy1['Index'].tolist()
delete_wellbore = []
for i in dummy2['Index']:
    delete_wellbore += i
dummy = dummy[~dummy['Index'].isin(delete_wellbore)].reset_index(drop=True)

dummy = dummy.drop(dummy[dummy['Дебит нефти, т/сут'] == 0].index)
            #n = 1
            #z = dummy['Пласт'].value_counts()[:n].index.tolist()
            #plast = str(z[0])
plast = 'Ю3'
dummy = dummy.drop(dummy[dummy['Пласт'] != plast].index).reset_index(
                drop=True)
dummy_res_zero = dummy.drop(dummy[dummy['Пластовое давление (ТР), атм'] != 0].index)
dummy_res_zero = dummy_res_zero.groupby('Скважина №').agg(
                {'Index': lambda x:
                x.tolist()})
dummy_res_zero['нули_пласт'] = dummy_res_zero.apply(lambda x: len(x['Index']), axis=1)
dummy_res_zero = dummy_res_zero.drop(dummy_res_zero[dummy_res_zero['нули_пласт'] < 3].index)
            # wells_del_both=dummy1['Index'].tolist()
delete_reservoir = []
for i in dummy_res_zero['Index']:
        delete_reservoir += i
dummy = dummy[~dummy['Index'].isin(delete_reservoir)].reset_index(drop=True)

dummy_bore_zero = dummy.drop(dummy[dummy['Забойное давление (ТР), атм'] != 0].index)
dummy_bore_zero = dummy_bore_zero.groupby('Скважина №').agg(
                {'Index': lambda x:
                x.tolist()})
dummy_bore_zero['нули_заб'] = dummy_bore_zero.apply(lambda x: len(x['Index']), axis=1)
dummy_bore_zero = dummy_bore_zero.drop(dummy_bore_zero[dummy_bore_zero['нули_заб'] < 3].index)
            # wells_del_both=dummy1['Index'].tolist()
delete_bore = []
for i in dummy_bore_zero['Index']:
    delete_bore += i
dummy = dummy[~dummy['Index'].isin(delete_bore)].reset_index(drop=True)

dummy_40 = dummy[dummy['Пластовое давление (ТР), атм'].isna()]
dummy_40 = dummy_40.groupby('Скважина №').agg(
                {'Index': lambda x:
                x.tolist()})
dummy_40['na_пласт'] = dummy_40.apply(lambda x: len(x['Index']), axis=1)
dummy_40 = dummy_40.drop(dummy_40[dummy_40['na_пласт'] < 3].index)
            # wells_del_both=dummy1['Index'].tolist()
del_plast = []
for i in dummy_40['Index']:
    del_plast += i
dummy = dummy[~dummy['Index'].isin(del_plast)].reset_index(drop=True)
dummy

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
0,2016-07-01,1,78.52,55.39,19.75,263.0,131.1,Ю3,135
1,2016-08-01,1,84.28,59.86,19.19,263.0,118.3,Ю3,136
2,2016-09-01,1,85.31,68.98,8.01,263.0,107.1,Ю3,137
3,2016-10-01,1,88.38,71.09,8.49,263.0,94.8,Ю3,138
4,2016-11-01,1,78.99,62.62,9.81,263.0,79.4,Ю3,139
...,...,...,...,...,...,...,...,...,...
21394,2021-10-01,949,84.51,29.30,60.56,205.0,194.8,Ю3,304410
21395,2021-11-01,949,83.47,28.11,61.68,205.0,180.8,Ю3,304411
21396,2021-12-01,949,80.92,25.38,64.32,205.0,173.8,Ю3,304412
21397,2022-01-01,949,81.83,24.15,66.43,268.0,166.8,Ю3,304413


In [4]:
dummy1 = dummy.groupby(target).agg(
                {'Дата': lambda x:
                x.tolist()}
            )
dummy1['История'] = dummy1.apply(
                lambda x:
                1 if len(x['Дата']) >= 6
                else 0, axis=1)
dummy1 = dummy1.drop(dummy1[dummy1['История'] != 1].index)
dummy1.reset_index(inplace=True)
a = dummy1['Скважина №'].tolist()
dummy = dummy[dummy['Скважина №'].isin(a)]
dummy

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
0,2016-07-01,1,78.52,55.39,19.75,263.0,131.1,Ю3,135
1,2016-08-01,1,84.28,59.86,19.19,263.0,118.3,Ю3,136
2,2016-09-01,1,85.31,68.98,8.01,263.0,107.1,Ю3,137
3,2016-10-01,1,88.38,71.09,8.49,263.0,94.8,Ю3,138
4,2016-11-01,1,78.99,62.62,9.81,263.0,79.4,Ю3,139
...,...,...,...,...,...,...,...,...,...
21388,2021-09-01,924,84.21,14.76,80.06,202.0,76.2,Ю3,296935
21389,2021-10-01,924,81.14,13.27,81.39,202.0,77.3,Ю3,296936
21390,2021-11-01,924,81.50,9.44,86.83,202.0,76.5,Ю3,296937
21391,2021-12-01,924,80.24,9.67,86.29,202.0,76.5,Ю3,296938


In [5]:
dummy["Пластовое давление (ТР), атм"] = dummy["Пластовое давление (ТР), атм"].replace(0, np.nan).bfill()
dummy["Забойное давление (ТР), атм"] = dummy["Забойное давление (ТР), атм"].replace(0, np.nan).bfill()
dummy['депрессия'] = dummy.apply(
                lambda x:
                x['Пластовое давление (ТР), атм'] - x['Забойное давление (ТР), атм'], axis=1)
dummy = dummy.drop(dummy[dummy['депрессия'] <= 0].index)
dummy = dummy.drop('депрессия', axis=1)
dummy

C:\Users\USER\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\USER\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\USER\miniconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
0,2016-07-01,1,78.52,55.39,19.75,263.0,131.1,Ю3,135
1,2016-08-01,1,84.28,59.86,19.19,263.0,118.3,Ю3,136
2,2016-09-01,1,85.31,68.98,8.01,263.0,107.1,Ю3,137
3,2016-10-01,1,88.38,71.09,8.49,263.0,94.8,Ю3,138
4,2016-11-01,1,78.99,62.62,9.81,263.0,79.4,Ю3,139
...,...,...,...,...,...,...,...,...,...
21388,2021-09-01,924,84.21,14.76,80.06,202.0,76.2,Ю3,296935
21389,2021-10-01,924,81.14,13.27,81.39,202.0,77.3,Ю3,296936
21390,2021-11-01,924,81.50,9.44,86.83,202.0,76.5,Ю3,296937
21391,2021-12-01,924,80.24,9.67,86.29,202.0,76.5,Ю3,296938


In [6]:
dummy.loc[dummy['Скважина №'] == '110']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index
188,2020-07-01,110,17.46,11.88,22.57,258.0,60.9,Ю3,9273
189,2020-08-01,110,16.66,11.78,19.56,258.0,53.3,Ю3,9274
190,2020-09-01,110,16.96,10.76,27.82,258.0,52.3,Ю3,9275
191,2020-10-01,110,16.71,11.46,21.98,208.0,47.2,Ю3,9276
192,2020-11-01,110,15.69,10.72,22.32,208.0,61.9,Ю3,9277
193,2020-12-01,110,15.05,10.52,20.49,208.0,49.9,Ю3,9278
194,2021-01-01,110,15.14,10.30,22.60,196.0,44.0,Ю3,9279
195,2021-02-01,110,15.24,10.56,21.16,196.0,42.0,Ю3,9280
196,2021-03-01,110,15.68,11.36,17.57,196.0,55.0,Ю3,9281
197,2021-04-01,110,15.52,11.41,16.36,196.0,59.0,Ю3,9282


In [7]:
data.loc[data['Скважина №'] == '110']

,Unnamed: 0,Скважина №,"Длина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после,ГС/ННС,Тип ГТМ,Пласт
479,479,110,817.866126,0.0,0,0,0,2020-02-17,ГС,ВНС,Ю3


In [8]:
data.loc[data['Скважина №'] == '1004_ГРП']

,Unnamed: 0,Скважина №,"Длина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после,ГС/ННС,Тип ГТМ,Пласт
3,3,1004_ГРП,0.0,1.0,100,4,100,2011-02-04,ННС+ГРП,ГРП,Ю3


In [9]:
data['для_грп'] = data.apply(
                lambda x:
                x['Скважина №'].partition('_')[0] if '_' in str(x['Скважина №'])
                else x['Скважина №'], axis=1)
data.loc[data['для_грп'] == '1004']

,Unnamed: 0,Скважина №,"Длина ГС, м",Число стадий ГРП,"Полудлина трещины, м","Ширина трещины, мм","Проницаемость проппанта, Д",Дата ВНР после,ГС/ННС,Тип ГТМ,Пласт,для_грп
3,3,1004_ГРП,0.0,1.0,100,4,100,2011-02-04,ННС+ГРП,ГРП,Ю3,1004
4,4,1004_ФРАК_1,0.0,1.0,100,4,100,2013-01-31,ННС+ГРП,ГРП,Ю3,1004
5,5,1004_ФРАК_2,0.0,1.0,100,4,100,2013-03-03,ННС+ГРП,ГРП,Ю3,1004


In [10]:
no_grp = data.groupby('для_грп').agg(
                {'ГС/ННС': lambda x:
                x.tolist()}
            )
no_grp.reset_index(inplace=True)
no_grp.loc[no_grp['для_грп'] == '1004']

,для_грп,ГС/ННС
2,1004,"[ННС+ГРП, ННС+ГРП, ННС+ГРП]"


In [11]:
no_grp['ГРП?'] = no_grp.apply(
                lambda x:
                0 if 'ГРП' not in str(x['ГС/ННС'])
                else 1, axis=1)
no_grp.loc[no_grp['для_грп'] == '1004']

,для_грп,ГС/ННС,ГРП?
2,1004,"[ННС+ГРП, ННС+ГРП, ННС+ГРП]",1


In [12]:
no_grp = no_grp.drop(no_grp[no_grp['ГРП?'] == 1].index)
no_grp.loc[no_grp['для_грп'] == '110']

,для_грп,ГС/ННС,ГРП?
9,110,[ГС],0


In [13]:
spisok = no_grp['для_грп'].tolist()
'110' in spisok

True

In [14]:
step = data.groupby('для_грп').agg(
                {date_column_name: lambda x:
                x.tolist()[0]
                if len(x.tolist()) == 1 else x.tolist()[1]}
            )
step

,Дата ВНР после
для_грп,
1,2016-03-12
10,2013-03-03
1004,2013-01-31
1005,2017-08-21
1006,2019-08-16
...,...
936,2008-09-27
942,2013-06-19
96,2019-07-31


In [15]:
step.reset_index(inplace=True)

In [16]:
step.loc[step['для_грп'] == '1004']

,для_грп,Дата ВНР после
2,1004,2013-01-31


In [17]:
step.loc[step['для_грп'] == '110']

,для_грп,Дата ВНР после
9,110,2020-02-17


In [18]:
step = step[~step['для_грп'].isin(spisok)].reset_index(drop=True)
step.loc[step['для_грп'] == '110']

,для_грп,Дата ВНР после


In [19]:
vvv = step['для_грп'].tolist()
vvv

['1',
 '10',
 '1004',
 '1005',
 '1006',
 '105',
 '1092',
 '11',
 '1111',
 '1112',
 '1113',
 '1124',
 '119',
 '12',
 '122',
 '1220',
 '1221',
 '1223',
 '1224',
 '123',
 '1230',
 '124',
 '1259',
 '1260',
 '1272',
 '128',
 '1291',
 '13',
 '130',
 '1302',
 '1303',
 '1304',
 '131',
 '1311',
 '1312',
 '1320',
 '133',
 '1334',
 '1341',
 '1344',
 '1345',
 '1349',
 '1366',
 '1367',
 '1383',
 '1384',
 '1385',
 '1386',
 '139',
 '14',
 '140',
 '141',
 '142',
 '143',
 '1433',
 '144',
 '146',
 '148',
 '149',
 '15',
 '150',
 '1500',
 '1501',
 '1502',
 '1503',
 '1504',
 '1505',
 '1507',
 '151',
 '1511',
 '1513',
 '1521',
 '1523',
 '153',
 '154',
 '1544',
 '1549',
 '1550',
 '1551',
 '1555',
 '1556',
 '1556Л',
 '1557',
 '1559',
 '1560',
 '1561',
 '1562',
 '158',
 '16',
 '1602',
 '1605',
 '1606',
 '1607',
 '1608',
 '1609',
 '1610',
 '1611',
 '162',
 '1620',
 '1621',
 '1622',
 '1623',
 '1625',
 '1626',
 '1628',
 '1630',
 '1631',
 '1632',
 '1633',
 '1634',
 '1635',
 '1636',
 '1637',
 '1638',
 '1639',
 '164

In [20]:
step['Скважина №'] = vvv
step = step.drop(['для_грп'], axis=1)
step.head(10)

,Дата ВНР после,Скважина №
0,2016-03-12,1
1,2013-03-03,10
2,2013-01-31,1004
3,2017-08-21,1005
4,2019-08-16,1006
5,2010-10-23,105
6,2010-07-21,1092
7,2009-04-01,11
8,2015-03-27,1111
9,2015-05-07,1112


In [21]:
dummy = dummy.merge(step, on=target, how='left')
dummy.head(10)

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index,Дата ВНР после
0,2016-07-01,1,78.52,55.39,19.75,263.0,131.1,Ю3,135,2016-03-12
1,2016-08-01,1,84.28,59.86,19.19,263.0,118.3,Ю3,136,2016-03-12
2,2016-09-01,1,85.31,68.98,8.01,263.0,107.1,Ю3,137,2016-03-12
3,2016-10-01,1,88.38,71.09,8.49,263.0,94.8,Ю3,138,2016-03-12
4,2016-11-01,1,78.99,62.62,9.81,263.0,79.4,Ю3,139,2016-03-12
5,2016-12-01,1,68.36,53.63,10.74,263.0,69.8,Ю3,140,2016-03-12
6,2017-01-01,1,57.02,42.88,14.44,236.0,83.5,Ю3,141,2016-03-12
7,2008-10-01,10,68.27,58.80,2.02,226.0,132.3,Ю3,244,2013-03-03
8,2008-11-01,10,68.07,58.57,2.12,226.0,117.6,Ю3,245,2013-03-03
9,2008-12-01,10,71.18,56.45,9.78,226.0,115.5,Ю3,246,2013-03-03


In [22]:
dummy.loc[dummy['Скважина №'] == '110']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index,Дата ВНР после
186,2020-07-01,110,17.46,11.88,22.57,258.0,60.9,Ю3,9273,NaT
187,2020-08-01,110,16.66,11.78,19.56,258.0,53.3,Ю3,9274,NaT
188,2020-09-01,110,16.96,10.76,27.82,258.0,52.3,Ю3,9275,NaT
189,2020-10-01,110,16.71,11.46,21.98,208.0,47.2,Ю3,9276,NaT
190,2020-11-01,110,15.69,10.72,22.32,208.0,61.9,Ю3,9277,NaT
191,2020-12-01,110,15.05,10.52,20.49,208.0,49.9,Ю3,9278,NaT
192,2021-01-01,110,15.14,10.30,22.60,196.0,44.0,Ю3,9279,NaT
193,2021-02-01,110,15.24,10.56,21.16,196.0,42.0,Ю3,9280,NaT
194,2021-03-01,110,15.68,11.36,17.57,196.0,55.0,Ю3,9281,NaT
195,2021-04-01,110,15.52,11.41,16.36,196.0,59.0,Ю3,9282,NaT


In [23]:
dummy.loc[dummy['Скважина №'] == '1004']

,Дата,Скважина №,"Дебит жидкости, м3/сут","Дебит нефти, т/сут","Обводненность, %","Пластовое давление (ТР), атм","Забойное давление (ТР), атм",Пласт,Index,Дата ВНР после
160,2012-01-01,1004,73.93,43.61,32.89,216.0,126.0,Ю3,889,2013-01-31
161,2012-02-01,1004,79.06,40.59,41.60,216.0,122.3,Ю3,890,2013-01-31
162,2012-03-01,1004,76.11,38.94,41.80,216.0,117.4,Ю3,891,2013-01-31
163,2012-04-01,1004,75.08,38.40,41.82,216.0,128.6,Ю3,892,2013-01-31
164,2012-05-01,1004,70.74,32.17,48.26,216.0,144.8,Ю3,893,2013-01-31
165,2012-06-01,1004,98.53,41.85,51.67,216.0,49.5,Ю3,894,2013-01-31
166,2012-07-01,1004,102.70,40.35,55.30,216.0,74.5,Ю3,895,2013-01-31
167,2012-08-01,1004,98.22,38.48,55.43,216.0,70.8,Ю3,896,2013-01-31
168,2012-09-01,1004,99.67,35.73,59.21,216.0,74.7,Ю3,897,2013-01-31
169,2012-10-01,1004,107.37,33.26,64.76,243.0,106.3,Ю3,898,2013-01-31
